# Platform 구분

In [ ]:

import platform
import os

def setup_by_os():    
    system_name = platform.system()

    if system_name == 'Darwin':  # macOS
        print(system_name)
    elif system_name == 'Linux':
        print(system_name)
    elif system_name == 'Windows':
        print(system_name)
    else:
        raise EnvironmentError(f"❌ 지원되지 않는 운영체제입니다: {system_name}")

# 한글 폰트 설정

In [ ]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

def set_font(): 
    # 폰트 설정
    font_path='/System/Library/Fonts/Supplemental/AppleGothic.ttf'
    fontprop = fm.FontProperties(fname=font_path, size=10)
    plt.rcParams['font.family'] = 'AppleGothic'  # macOS
    plt.rcParams['axes.unicode_minus'] = False
    
set_font()

# PyTorch GPU 설정

In [ ]:
import torch

def get_device():
    # Determine best available device (MPS > CUDA > CPU)
    if torch.backends.mps.is_available():
        device = torch.device("mps")  # Mac M-series GPU
    elif torch.cuda.is_available():
        device = torch.device("cuda")  # NVIDIA GPU
    else:
        device = torch.device("cpu")   # CPU fallback
    return device

## a_my_rag_module 사용하기 위한 경로설정

In [ ]:
import os, sys

# a_my_rag_module를 사용하기 위해 프로젝트 루트 경로를 sys.path에 추가
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# 전처리

## 수치형과 범주형 컬럼 분리

In [ ]:
import pandas as pd
# 수치형과 범주형 컬럼 분리
numeric_cols = df_original.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_original.select_dtypes(include=['object']).columns.tolist()

## 범주형 컬럼 지정 숫자로 변환

In [ ]:
activity_mapping = {
    'light': 1,
    'moderate': 2, 
    'intense': 3
}
df_copy['activity'] = df_copy['activity'].map(activity_mapping)

## 데이터 스케일링

In [ ]:
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 각 컬럼의 데이터 분포 시각화

In [ ]:
# 각 컬럼의 데이터 분포 시각화
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 원본 데이터프레임 사용 (인코딩 전)
df_original = df.copy()

# 수치형과 범주형 컬럼 분리
numeric_cols = df_original.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_original.select_dtypes(include=['object']).columns.tolist()

print(f"수치형 컬럼 ({len(numeric_cols)}개): {numeric_cols}")
print(f"범주형 컬럼 ({len(categorical_cols)}개): {categorical_cols}")

# 1. 수치형 변수 분포 시각화
if numeric_cols:
    n_numeric = len(numeric_cols)
    cols = 3
    rows = (n_numeric + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 4*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i, col in enumerate(numeric_cols):
        row = i // cols
        col_idx = i % cols
        
        # 히스토그램과 KDE
        axes[row, col_idx].hist(df_original[col], bins=30, alpha=0.7, density=True, color='skyblue')
        axes[row, col_idx].set_title(f'{col} 분포')
        axes[row, col_idx].set_xlabel(col)
        axes[row, col_idx].set_ylabel('Density')
        axes[row, col_idx].grid(alpha=0.3)
        
        # 기본 통계 정보 텍스트 추가
        mean_val = df_original[col].mean()
        std_val = df_original[col].std()
        axes[row, col_idx].axvline(mean_val, color='red', linestyle='--', alpha=0.8, label=f'평균: {mean_val:.1f}')
        axes[row, col_idx].legend()
    
    # 빈 서브플롯 제거
    for i in range(n_numeric, rows * cols):
        row = i // cols
        col_idx = i % cols
        axes[row, col_idx].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('수치형 변수 분포', fontsize=16, y=1.02)
    plt.show()

# 2. 범주형 변수 분포 시각화
if categorical_cols:
    n_categorical = len(categorical_cols)
    cols = 2
    rows = (n_categorical + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(12, 4*rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    elif n_categorical == 1:
        axes = [axes]
    
    for i, col in enumerate(categorical_cols):
        row = i // cols
        col_idx = i % cols
        
        # 막대그래프
        value_counts = df_original[col].value_counts()
        bars = axes[row, col_idx].bar(range(len(value_counts)), value_counts.values, color='lightcoral', alpha=0.7)
        axes[row, col_idx].set_xticks(range(len(value_counts)))
        axes[row, col_idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
        axes[row, col_idx].set_title(f'{col} 분포')
        axes[row, col_idx].set_ylabel('Count')
        axes[row, col_idx].grid(axis='y', alpha=0.3)
        
        # 막대 위에 개수 표시
        for bar, count in zip(bars, value_counts.values):
            axes[row, col_idx].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                                   str(count), ha='center', va='bottom')
    
    # 빈 서브플롯 제거
    for i in range(n_categorical, rows * cols):
        row = i // cols
        col_idx = i % cols
        if rows > 1:
            axes[row, col_idx].set_visible(False)
        else:
            axes[col_idx].set_visible(False)
    
    plt.tight_layout()
    plt.suptitle('범주형 변수 분포', fontsize=16, y=1.02)
    plt.show()

# 3. 기본 통계 정보 출력
print("\n" + "="*80)
print("데이터셋 기본 정보")
print("="*80)
print(f"전체 샘플 수: {len(df_original)}")
print(f"전체 컬럼 수: {len(df_original.columns)}")

# 결측값 확인
missing_values = df_original.isnull().sum()
missing_columns = missing_values[missing_values > 0]

print(f"전체 결측값 개수: {missing_values.sum()}")
if len(missing_columns) > 0:
    print("결측값이 있는 컬럼:")
    for col, count in missing_columns.items():
        percentage = (count / len(df_original)) * 100
        print(f"  - {col}: {count}개 ({percentage:.1f}%)")
else:
    print("결측값이 있는 컬럼: 없음")

print("\n수치형 변수 기본 통계:")
print("-" * 50)
print(df_original[numeric_cols].describe().round(2))

print("\n범주형 변수 분포:")
print("-" * 50)
for col in categorical_cols:
    print(f"\n{col}:")
    print(df_original[col].value_counts())
    
# 4. Heart Disease 분포 (타겟 변수)
if 'Heart Disease' in df_original.columns:
    plt.figure(figsize=(8, 5))
    target_counts = df_original[target_col].value_counts()
    
    # 파이 차트
    plt.subplot(1, 2, 1)
    plt.pie(target_counts.values, labels=['No Heart Disease', target_col], 
            autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
    plt.title('Heart Disease 분포 (파이차트)')
    
    # 막대 차트
    plt.subplot(1, 2, 2)
    bars = plt.bar(['No (0)', 'Yes (1)'], target_counts.values, color=['lightgreen', 'lightcoral'], alpha=0.7)
    plt.title('Heart Disease 분포 (막대차트)')
    plt.ylabel('Count')
    
    # 막대 위에 개수와 비율 표시
    total = target_counts.sum()
    for bar, count in zip(bars, target_counts.values):
        percentage = count / total * 100
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, 
                f'{count}\n({percentage:.1f}%)', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nHeart Disease 분포:")
    print(f"정상 (0): {target_counts[0]}명 ({target_counts[0]/total*100:.1f}%)")
    print(f"심장병 (1): {target_counts[1]}명 ({target_counts[1]/total*100:.1f}%)")

# Target과의 상관관계 분석

In [ ]:
# Heart Disease 상관관계 분석
import matplotlib.pyplot as plt
import seaborn as sns


target_col = 'Heart Disease'

# 전체 데이터셋에 타겟 변수 포함하여 상관관계 계산
df_corr = X.copy()
df_corr[target_col] = y

# 상관관계 행렬 계산
correlation_matrix = df_corr.corr()

# 1. 전체 상관관계 히트맵
plt.figure(figsize=(14, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='RdBu_r', center=0, 
            square=True, fmt='.2f', cbar_kws={'label': 'Correlation Coefficient'})
plt.title('Heart Disease Dataset - Feature Correlation Matrix', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

# 2. Heart Disease와의 상관관계만 따로 시각화
heart_disease_corr = correlation_matrix[target_col].drop(target_col).sort_values(key=abs, ascending=False)

plt.figure(figsize=(10, 8))
colors = ['red' if x < 0 else 'blue' for x in heart_disease_corr.values]
bars = plt.barh(range(len(heart_disease_corr)), heart_disease_corr.values, color=colors, alpha=0.7)
plt.yticks(range(len(heart_disease_corr)), heart_disease_corr.index)
plt.xlabel('Correlation with Heart Disease')
plt.title('Feature Correlation with Heart Disease', fontsize=14, pad=20)
plt.grid(axis='x', alpha=0.3)

# 막대 위에 상관계수 값 표시
for i, (bar, value) in enumerate(zip(bars, heart_disease_corr.values)):
    plt.text(value + (0.01 if value >= 0 else -0.01), i, f'{value:.3f}', 
             va='center', ha='left' if value >= 0 else 'right', fontweight='bold')

plt.tight_layout()
plt.show()

# 3. 상관관계 수치 출력
print("Heart Disease와 각 특성간의 상관관계:")
print("=" * 50)
for feature, corr_value in heart_disease_corr.items():
    direction = "양의 상관관계" if corr_value > 0 else "음의 상관관계"
    strength = "강한" if abs(corr_value) > 0.5 else "중간" if abs(corr_value) > 0.3 else "약한"
    print(f"{feature:25} : {corr_value:6.3f} ({strength} {direction})")